# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherDF = pd.read_csv('../Output/weather_data_df.csv')
weatherDF.head()


,City,Lat,Lon,Max Temp,Humidity,Wind,Cloudiness,Country
0,Leningradskiy,69.38,178.42,34.54,88,14.90,100,RU
1,Tasiilaq,65.61,-37.64,42.80,87,14.99,100,GL
2,Port-Gentil,-0.72,8.78,78.80,78,10.29,40,GA
3,Ilulissat,69.22,-51.10,37.40,51,2.24,75,GL
4,Hobart,-42.88,147.33,45.00,70,8.05,20,AU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = weatherDF[['Lat', 'Lon']]
fig = gmaps.figure(zoom_level=2, center=(0,0))
heat_layer = gmaps.heatmap_layer(locations, weights = weatherDF.Humidity,
                                dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
niceDF = weatherDF.loc[(weatherDF['Max Temp'] > 70) & (weatherDF['Max Temp'] < 80)\
                          & (weatherDF['Humidity'] < 50)\
                          & (weatherDF['Cloudiness'] < 20)\
                          & (weatherDF['Wind'] < 10), :]
niceDF.head(10)

,City,Lat,Lon,Max Temp,Humidity,Wind,Cloudiness,Country
40,Albany,42.60,-73.97,73.00,45,9.31,7,US
85,Sakakah,29.97,40.21,73.40,26,5.82,0,SA
99,Kutum,14.20,24.67,79.70,18,9.17,0,SD
120,Karratha,-20.74,116.85,73.26,39,8.39,0,AU
320,Fernley,39.61,-119.25,73.00,14,1.01,0,US
396,Vallenar,-28.57,-70.76,77.31,33,8.03,0,CL
409,Upington,-28.45,21.26,73.40,12,1.12,0,ZA
511,Punta Alta,-38.88,-62.07,74.55,37,7.58,0,AR
527,San Julián,21.02,-102.17,77.77,38,6.02,4,MX
554,Roebourne,-20.78,117.13,75.18,38,7.72,0,AU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = niceDF
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

C:\Users\JCCla\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lon,Max Temp,Humidity,Wind,Cloudiness,Country,Hotel Name
40,Albany,42.60,-73.97,73.00,45,9.31,7,US,
85,Sakakah,29.97,40.21,73.40,26,5.82,0,SA,
99,Kutum,14.20,24.67,79.70,18,9.17,0,SD,
120,Karratha,-20.74,116.85,73.26,39,8.39,0,AU,
320,Fernley,39.61,-119.25,73.00,14,1.01,0,US,
396,Vallenar,-28.57,-70.76,77.31,33,8.03,0,CL,
409,Upington,-28.45,21.26,73.40,12,1.12,0,ZA,
511,Punta Alta,-38.88,-62.07,74.55,37,7.58,0,AR,
527,San Julián,21.02,-102.17,77.77,38,6.02,4,MX,
554,Roebourne,-20.78,117.13,75.18,38,7.72,0,AU,


In [7]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {'key': g_key, 'radius': 5000, 'type': 'lodging', 'keyword': 'hotel'}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lon']
    params['location'] = f'{lat},{lng}'
    response = requests.get(url, params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except:
        pass

C:\Users\JCCla\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
hotel_df.head(10)

,City,Lat,Lon,Max Temp,Humidity,Wind,Cloudiness,Country,Hotel Name
40,Albany,42.60,-73.97,73.00,45,9.31,7,US,
85,Sakakah,29.97,40.21,73.40,26,5.82,0,SA,Raoum Inn Hotel
99,Kutum,14.20,24.67,79.70,18,9.17,0,SD,
120,Karratha,-20.74,116.85,73.26,39,8.39,0,AU,Karratha International Hotel
320,Fernley,39.61,-119.25,73.00,14,1.01,0,US,Comfort Suites Fernley
396,Vallenar,-28.57,-70.76,77.31,33,8.03,0,CL,Hotel Cecil
409,Upington,-28.45,21.26,73.40,12,1.12,0,ZA,Protea Hotel by Marriott Upington
511,Punta Alta,-38.88,-62.07,74.55,37,7.58,0,AR,Hotel Barcelona Center - Restaurant & Confitería
527,San Julián,21.02,-102.17,77.77,38,6.02,4,MX,Hotel Casa Grande
554,Roebourne,-20.78,117.13,75.18,38,7.72,0,AU,Latitude20 Roebourne Village


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info) 
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))